In [1]:
import pandas as pd

### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [4]:
ratings = pd.read_csv('ratings_small.csv')

In [6]:
def classify_rating(rating):
  if rating <= 2:
    return 'Низкий рейтинг'
  
  if rating <= 4:
    return 'Средний рейтинг'
  
  if rating >= 4.5:
    return 'Высокий рейтинг'
  
  return 'Рейтинг не определен'

In [7]:
ratings['class'] = ratings['rating'].apply(classify_rating)

In [10]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,Средний рейтинг
1,1,1029,3.0,1260759179,Средний рейтинг
2,1,1061,3.0,1260759182,Средний рейтинг
3,1,1129,2.0,1260759185,Низкий рейтинг
4,1,1172,4.0,1260759205,Средний рейтинг


### Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

```
geo_data = {
  'Центр': ['москва', 'тула', 'ярославль'],
  'Северо-Запад': ['петербург', 'псков', 'мурманск'],
  'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```


Результат классификации запишите в отдельный столбец region.

In [11]:
keywords = pd.read_csv('keywords.csv')

In [12]:
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [31]:
def is_keyword_contains_any_of_cities(source_string, cities):
  for city in cities:
    if city.lower() in source_string.lower():
      return True
    
  return False

def get_region(keyword):
  geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
  }
  
  for region, cities in geo_data.items():
    if is_keyword_contains_any_of_cities(keyword, cities):
      return region
  
  return 'undefined'

In [33]:
keywords['region'] = keywords['keyword'].apply(get_region)

In [42]:
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [44]:
# проверка
keywords[keywords.region != 'undefined'].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


### Задание 3

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    1. для каждой строки пройдите по всем годам списка years
    2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
    
    
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [213]:
movies = pd.read_csv('movies.csv')

In [214]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [219]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,Средний рейтинг
1,1,1029,3.0,1260759179,Средний рейтинг
2,1,1061,3.0,1260759182,Средний рейтинг
3,1,1129,2.0,1260759185,Низкий рейтинг
4,1,1172,4.0,1260759205,Средний рейтинг


In [161]:
years = list(range(1950, 2011))

In [211]:
def production_year(movie_title):
  for year in years:
    if str(year) in movie_title:
      return year
    
  return 1900

In [238]:
movies['year'] = movies['title'].apply(production_year)

In [239]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [240]:
ratings_with_years = ratings.join(movies, on='movieId', rsuffix='_movies')

In [254]:
ratings_with_years.head()

,userId,movieId,rating,timestamp,class,movieId_movies,title,genres,year
0,1,31,2.5,1260759144,Средний рейтинг,32.0,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,1995.0
1,1,1029,3.0,1260759179,Средний рейтинг,1053.0,Normal Life (1996),Crime|Drama|Romance,1996.0
2,1,1061,3.0,1260759182,Средний рейтинг,1088.0,Dirty Dancing (1987),Drama|Musical|Romance,1987.0
3,1,1129,2.0,1260759185,Низкий рейтинг,1157.0,"Symphonie pastorale, La (1946)",Drama,1900.0
4,1,1172,4.0,1260759205,Средний рейтинг,1202.0,Withnail & I (1987),Comedy,1987.0


In [257]:
ratings_with_years.groupby('year').mean().sort_values(by='rating', ascending=False)

,userId,movieId,rating,timestamp,movieId_movies
year,,,,,
1970.0,351.266362,4820.041096,3.886606,1.127262e+09,10529.878234
1967.0,347.325843,4660.841448,3.823346,1.124705e+09,12102.630462
1955.0,348.608073,4781.998698,3.805339,1.155352e+09,11857.720052
1963.0,330.416520,5194.476274,3.791740,1.136261e+09,9789.414763
1952.0,339.600683,5772.218430,3.773038,1.160539e+09,13056.754266
...,...,...,...,...,...
1961.0,344.400685,4849.253425,3.357877,1.091393e+09,10218.565068
1953.0,374.316038,11807.745283,3.299528,1.143191e+09,38501.183962
1988.0,337.946823,4726.618094,3.292127,1.124661e+09,9333.621547
